This tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2025.2_tutorials/wallaroo-model-operations-tutorials/deploy/by-framework/keras).

## Wallaroo Model Upload via the Wallaroo SDK: TensorFlow keras Sequential Single IO

The following tutorial demonstrates how to upload a TensorFlow keras Sequential Single IO model to a Wallaroo instance.

### Tutorial Goals

Demonstrate the following:

* Upload a TensorFlow keras Sequential Single IO to a Wallaroo instance.
* Create a pipeline and add the model as a pipeline step.
* Perform a sample inference.

### Prerequisites

* Wallaroo Version 2023.2.1 or above instance.

### References

* [Wallaroo MLOps API Essentials Guide: Model Upload and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-api-essential-guide/wallaroo-mlops-api-essentials-guide-model-uploads/)
* [Wallaroo API Connection Guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-connection-guide/)
* [DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.

In [1]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline   import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework
from wallaroo.object import EntityNotFoundError

import pyarrow as pa
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

import datetime

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more details on logging in through Wallaroo, see the [Wallaroo SDK Essentials Guide: Client Connection](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [ ]:
wl = wallaroo.Client()

### Set Variables

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.

In [3]:
workspace_name = f'keras-sequential-single-io'
pipeline_name = f'keras-sequential-single-io'

model_name = 'keras-sequential-single-io'
model_file_name = 'models/model-auto-conversion_keras_single_io_keras_sequential_model.h5'

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Configure Data Schemas

The following parameters are required for TensorFlow keras models.  Note that while some fields are considered as **optional** for the `upload_model` method, they are required for proper uploading of a TensorFlow Keras model to Wallaroo.

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Upload Method Optional, TensorFlow keras model Required*) | Set as the `Framework.KERAS`. |
|`input_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, TensorFlow Keras model Required*) | The input schema in Apache Arrow schema format. |
|`output_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, TensorFlow Keras model Required*) | The output schema in Apache Arrow schema format. |
| `convert_wait` | `bool` (*Upload Method Optional, TensorFlow model Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

In [5]:
input_schema = pa.schema([
    pa.field('input', pa.list_(pa.float64(), list_size=10))
])
output_schema = pa.schema([
    pa.field('output', pa.list_(pa.float64(), list_size=32))
])

### Upload Model

The model will be uploaded with the framework set as `Framework.KERAS`.

In [6]:
framework=Framework.KERAS

model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=framework, 
                        input_schema=input_schema, 
                        output_schema=output_schema)
model

Waiting for model loading - this will take up to 10min.

Model is pending loading to a native runtime.
Model is attempting loading to a native runtime...
Incompatible
Model is pending loading to a container runtime..
Model is attempting loading to a container runtime..........
Successful
Ready


Name,keras-sequential-single-io
Version,88bca1de-23a6-43a4-b18a-2bbac0c2f1f4
File Name,model-auto-conversion_keras_single_io_keras_sequential_model.h5
SHA,f7e49538e38bebe066ce8df97bac8be239ae8c7d2733e500c8cd633706ae95a8
Status,ready
Image Path,proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2025.1.2-6307
Architecture,x86
Acceleration,none
Updated At,2025-13-Aug 14:38:54
Workspace id,1451
Workspace name,keras-sequential-single-io


In [7]:
model.config().runtime()

'flight'

### Deploy Pipeline

The model is uploaded and ready for use.  We'll add it as a step in our pipeline, then deploy the pipeline.  For this example we're allocated 0.25 cpu and 4 Gi RAM to the pipeline through the pipeline's deployment configuration.

In [8]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [ ]:
# clear the pipeline if used in a previous tutorial
pipeline.undeploy()
pipeline.clear()
pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config, wait_for_status=False)

In [11]:
import time
time.sleep(15)

while pipeline.status()['status'] != 'Running':
    time.sleep(15)
    print("Waiting for deployment.")
    pipeline.status()['status']
pipeline.status()['status']


Waiting for deployment.
Waiting for deployment.
Waiting for deployment.


'Running'

### Run Inference

A sample inference will be run.  First the pandas DataFrame used for the inference is created, then the inference run through the pipeline's `infer` method.

In [12]:
input_data = np.random.rand(10, 10)
mock_dataframe = pd.DataFrame({
    "input": input_data.tolist()
})
mock_dataframe

,input
0,"[0.5000553806200765, 0.16702887890972484, 0.61..."
1,"[0.3282489991940838, 0.8423481674759563, 0.146..."
2,"[0.2818252947184633, 0.4844710998679015, 0.303..."
3,"[0.14775350358737516, 0.8864875137773521, 0.10..."
4,"[0.48171129745527297, 0.9512782221706774, 0.75..."
5,"[0.2012064897721495, 0.00851796175669739, 0.73..."
6,"[0.22517226845525729, 0.40472609779152746, 0.1..."
7,"[0.585200431533809, 0.2996441899149356, 0.4810..."
8,"[0.5916241325754229, 0.2506798677381258, 0.282..."
9,"[0.1234836874686801, 0.34248920770381186, 0.15..."


In [13]:
pipeline.infer(mock_dataframe)

,time,in.input,out.output,anomaly.count
0,2025-08-13 14:42:36.834,"[0.5000553806, 0.1670288789, 0.6142694366, 0.3...","[0.03336078301072121, 0.0260093342512846, 0.04...",0
1,2025-08-13 14:42:36.834,"[0.3282489992, 0.8423481675, 0.1462019543, 0.6...","[0.03355403617024422, 0.019938213750720024, 0....",0
2,2025-08-13 14:42:36.834,"[0.2818252947, 0.4844710999, 0.3032319972, 0.3...","[0.020366527140140533, 0.029036473482847214, 0...",0
3,2025-08-13 14:42:36.834,"[0.1477535036, 0.8864875138, 0.1017666358, 0.6...","[0.038360852748155594, 0.02053960971534252, 0....",0
4,2025-08-13 14:42:36.834,"[0.4817112975, 0.9512782222, 0.7577658298, 0.3...","[0.03668661043047905, 0.022566303610801697, 0....",0
5,2025-08-13 14:42:36.834,"[0.2012064898, 0.0085179618, 0.7336150214, 0.3...","[0.031458839774131775, 0.02952795848250389, 0....",0
6,2025-08-13 14:42:36.834,"[0.2251722685, 0.4047260978, 0.1733992036, 0.6...","[0.025476163253188133, 0.026227081194519997, 0...",0
7,2025-08-13 14:42:36.834,"[0.5852004315, 0.2996441899, 0.4810742768, 0.8...","[0.029971962794661522, 0.03165528178215027, 0....",0
8,2025-08-13 14:42:36.834,"[0.5916241326, 0.2506798677, 0.2828436536, 0.0...","[0.022488953545689583, 0.026670964434742928, 0...",0
9,2025-08-13 14:42:36.834,"[0.1234836875, 0.3424892077, 0.1523391184, 0.1...","[0.023797933012247086, 0.028810342773795128, 0...",0


### Undeploy Pipelines

With the tutorial complete, the pipeline is undeployed to return the resources back to the cluster.

In [15]:
pipeline.undeploy()

name,keras-sequential-single-io
created,2025-01-06 21:09:30.713115+00:00
last_updated,2025-08-13 14:39:01.505298+00:00
deployed,False
workspace_id,1451
workspace_name,keras-sequential-single-io
arch,x86
accel,none
tags,
versions,"c2897940-07cf-49e6-aa64-42d66c19cc1b, 46f701f5-d234-4d24-81f8-45934f57f553, 40ad9f0f-1971-4a8f-9e32-6b64a38e9d82, 4a7fb6df-2465-4243-ba35-7647a2d75774, 9bf13dbf-fc77-41b0-9d8e-d2255e173daf, e2f236a8-efc7-4029-bf64-23e888c485d1, a609d3d7-fae5-472e-90f6-08f5933d3364, 7c76b0e7-3a2e-4c50-a1a8-0a828740a1be, 47daa201-cf92-4e58-bd79-fb69ab8ed0b9, d747800b-2e0c-44bc-b084-b93e0e6d9131, 5953694b-21cf-4619-ad9b-6a15f657d209, fe0b5086-6185-4d5d-819d-6ff96df77b4b, 34cc7983-dca3-43db-bf22-20a508b76cbe, 26998745-44db-46ae-b45d-5f0e5481a211, b617fca8-2b86-41ff-b0b8-eebf5aa41d70"
steps,keras-sequential-single-io
